# Start lokign at eye-tracking data from risk task

1. clean up data, put into right format!

* renamed files (locally, in this script), given dates --> `only one file per sub` with sufficient size... is it only from one format, and if so, which??
* on scloud: `prep_eyetrack_data.py`, edf2asc, then get_saccades & messages (not sure the later with right columns)

--> checked: it seems like the file responds to the `symbolic format` of the task! {compared the messages.tsv file created like above with the events.tsv files}
(which is good?! as we find the interesting behavioral effect for the symbolic format....) 
--> I should verify that this is the case for all subjects (also a way to check that the file renaming worked correctly!)

### from Gilles

This is a library that Tomas and Jan-Willem and I built to analyze data from the Eyelink.
https://github.com/Gilles86/hedfpy

These are some analyses on Miguel's data using that stuff.
https://github.com/Gilles86/riskeye

Careful, you need the edf2asc-binary (computer program). I think for that you need officially need a Eyelink account.

And look for the 'EyeLink Developer’s Kit'

But you can also find a binary in the root directory of this repo
https://github.com/Gilles86/riskeye

I think it's for Linux/Ubuntu, so you can run it on Sciencecloud or Docker.

### file conversion

on sciencecloud `~/git/riskeye/edf2asc`
created `/mnt_03/ds-dnumrisk/09_14_11.asc`

## renaming to organize

* there are `.edf` files in the `behavior_risk/sub-XX..` folders, but very small, (bytes to KB, but not ~ 5MB as in the proper transfered one from the magJudge task)

* from eyetracking computer, there is a folder with files from each date there was a study participant, but more than 2 (cause there were 2 presentation formats!) - & not 4 (cause of double visit that day) - and not alle of them with the approproate file size .....



In [83]:
import os
import pandas as pd
import os.path as op
from datetime import datetime
import numpy as np

bids_folder = '/Users/mrenke/data/ds-dnumrisk'
pati_eye = op.join(bids_folder, 'sourcedata/eyetracking_risk/eyetracking_risk_original')
pati_eye_new = op.join(bids_folder, 'sourcedata/eyetracking_risk/eyetracking_risk_renamed')
#os.makedirs(pati_eye_new, exist_ok=True)

fns = os.listdir(pati_eye)

In [2]:
scan_table = op.join(bids_folder, 'add_tables/subjects_recruit&scan_scanned_final.csv')
df = pd.read_csv(scan_table, header=1, usecols=[0,1,2,3,4],nrows=66)#.dropna() #,#sheet_name='scanned', header=1)  
df['date'] = pd.to_datetime(df['date'],dayfirst=True)
df['subject ID'] = df['subject ID'].astype(int)
#df.set_index('subject ID', inplace=True)|
df


,subject ID,group,name,gender,date
0,1,0,Lucie Allenman,w,2023-02-06
1,2,1,Sophia Atzori,w,2023-02-16
2,3,0,Donika Z...,w,2023-02-16
3,4,1,Carlota Vega,w,2023-02-24
4,5,0,Hanna Weissberger,w,2023-02-24
...,...,...,...,...,...
61,62,0,Bigna von der Heiden,w,2024-04-12
62,63,0,Youssouf Abdellaoui,m,2024-04-15
63,64,0,Aron Karrer,m,2024-04-18
64,65,0,Andrea Brun,w,2024-05-03


In [3]:
# --> lets make an overview table of all the files in the directory

df_sum = pd.DataFrame(columns=['filename','file_size','date','daytime_h','daytime_m','subject ID'])
for fn in fns:
    full_fn = os.path.join(pati_eye,fn) 

    file_size = os.path.getsize(full_fn)
    mod_time = os.path.getmtime(full_fn)
    mod_time = datetime.fromtimestamp(mod_time)
    matching_row = df[df['date'].apply(lambda x: (x.year == mod_time.year) and (x.month == mod_time.month) and (x.day == mod_time.day))]

    daytime_h = mod_time.hour 
    daytime_m = mod_time.minute
    subject_ID = matching_row['subject ID'].values if not matching_row.empty else None

    new_row = pd.DataFrame([{'filename': fn, 'file_size': file_size, 'date': mod_time.strftime('%d-%m-%Y'), 'daytime_h': daytime_h, 'daytime_m': daytime_m, 'subject ID': subject_ID}])
    df_sum = pd.concat([df_sum, new_row], ignore_index=True)
    #df_sum = df_sum.append({'filename':fn, 'file_size':file_size, 'mod_time':mod_time, 'daytime_h':daytime_h, 'daytime_m':daytime_m, 'subject ID':subject_ID}, ignore_index=True)



In [76]:
# filter the table for files with sufficient size
size_thresh = 4 * 1000000 # 1 MB
df_sum_fil = df_sum[df_sum['file_size']> size_thresh]  # only files larger than 4 MB
# df_sum_fil.sort_values(by='date', ascending=False).to_csv(op.join(bids_folder, 'sourcedata','overview_table_eyetrackingData.csv'), index=False)
df_sum_fil_fil = df_sum_fil[~df_sum_fil['filename'].str.match(r'^[A-Za-z]')] # remove weird files (all beli=ong to two subjects..., see overview_table_eyetrackingData)
df_sum_fil_fil # 67 subs! one too much but otherwise looks good... apart from the different formats...

,filename,file_size,date,daytime_h,daytime_m,subject ID
2,11_32_33.edf,6849498,01-03-2024,9,22,"[54, 55]"
4,12_03_33.edf,5554645,12-07-2023,10,1,"[34, 35]"
6,10_54_48.edf,5174266,12-04-2023,8,56,[12]
10,16_23_07.edf,5231303,15-03-2023,14,26,[11]
11,17_12_44.edf,5722832,16-02-2024,15,7,[52]
...,...,...,...,...,...,...
168,15_45_24.edf,6509370,21-06-2023,13,46,[32]
169,16_00_00.edf,4942023,17-07-2023,13,57,[36]
177,11_04_02.edf,6284504,16-02-2023,9,11,"[2, 3]"
183,15_08_53.edf,5955882,20-06-2023,13,8,"[29, 30, 31]"


### problem of double scan days
differentiate between 
* double session (Caroline helped, so first ID-number should have the risk session later)
* two sesssion on one day, first ID also has earlier time

In [77]:
import numpy as np
df_sum_fil_fil['assignedID'] = np.NaN

for index, row in df_sum_fil_fil.iterrows():
    if  pd.isna(row['assignedID']):
        if len(row['subject ID']) == 1:
            df_sum_fil_fil.at[index, 'assignedID'] = row['subject ID'][0]              
        if len(row['subject ID']) == 2:
            tmp = df_sum_fil_fil[df_sum_fil_fil['date'] == row['date']]
            tmp = tmp.reset_index()#.loc[1]
            index_otherFile_sameDate = tmp['index'][1]
            date = row['date']

            if np.abs(tmp['daytime_h'][0] - tmp['daytime_h'][1]) > 2 : # separate session
                #print(f'subsequent session on {date}')
                if tmp['daytime_h'][0] < tmp['daytime_h'][1]: # whichever was aquired first has the smaller ID
                    df_sum_fil_fil.at[index, 'assignedID'] = row['subject ID'][0] # index of current file (row) - first in tmp df ([0]) - gets forst sub ID (smaller one)
                    df_sum_fil_fil.at[index_otherFile_sameDate, 'assignedID'] = row['subject ID'][1]
                else:
                    df_sum_fil_fil.at[index, 'assignedID'] = row['subject ID'][1] # the other way around from above
                    df_sum_fil_fil.at[index_otherFile_sameDate, 'assignedID'] = row['subject ID'][0]

            else:  # double session - the other way around as above, whichever was aquired first has the larger ID
                #print(f'double session on {date}')
                if tmp['daytime_h'][0] < tmp['daytime_h'][1]: # whichever was aquired first has the larger ID
                    df_sum_fil_fil.at[index, 'assignedID'] = row['subject ID'][1] 
                    df_sum_fil_fil.at[index_otherFile_sameDate, 'assignedID'] = row['subject ID'][0]
                else:
                    df_sum_fil_fil.at[index, 'assignedID'] = row['subject ID'][0] # the other way around from above
                    df_sum_fil_fil.at[index_otherFile_sameDate, 'assignedID'] = row['subject ID'][1]

# hard code the 3 sessions on one day:
df_sum_fil_fil.at[45, 'assignedID'] = 29 # at 9 am - single session gets first ID
df_sum_fil_fil.at[141 , 'assignedID'] = 31 # at 13:40 - part of double session
df_sum_fil_fil.at[183 , 'assignedID'] =  30 # at 13:08
                       

df_sum_fil_fil

/var/folders/3k/8g0xv78x051fznwyh_m5xcn8f91w3q/T/ipykernel_40870/1161902817.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sum_fil_fil['assignedID'] = np.NaN


,filename,file_size,date,daytime_h,daytime_m,subject ID,assignedID
2,11_32_33.edf,6849498,01-03-2024,9,22,"[54, 55]",54.0
4,12_03_33.edf,5554645,12-07-2023,10,1,"[34, 35]",34.0
6,10_54_48.edf,5174266,12-04-2023,8,56,[12],12.0
10,16_23_07.edf,5231303,15-03-2023,14,26,[11],11.0
11,17_12_44.edf,5722832,16-02-2024,15,7,[52],52.0
...,...,...,...,...,...,...,...
168,15_45_24.edf,6509370,21-06-2023,13,46,[32],32.0
169,16_00_00.edf,4942023,17-07-2023,13,57,[36],36.0
177,11_04_02.edf,6284504,16-02-2023,9,11,"[2, 3]",2.0
183,15_08_53.edf,5955882,20-06-2023,13,8,"[29, 30, 31]",30.0


In [71]:
# check: example: 12-07-2023 - double session, 01-03-2024 - separate sessions
df_sum_fil_fil[df_sum_fil_fil['date'] == '12-07-2023'] 

,filename,file_size,date,daytime_h,daytime_m,subject ID,assignedID
4,12_03_33.edf,5554645,12-07-2023,10,1,"[34, 35]",34.0
143,10_40_08.edf,7255874,12-07-2023,8,41,"[34, 35]",35.0


In [74]:
#example with 3 subs one day: 
df_sum_fil_fil[df_sum_fil_fil['date'] == '20-06-2023']

# earliest fn - sub-29
# later are closer in time, here middle one should be 31 and last one 30

,filename,file_size,date,daytime_h,daytime_m,subject ID,assignedID
45,11_00_53.edf,8018070,20-06-2023,9,3,"[29, 30, 31]",29.0
141,15_41_20.edf,5771555,20-06-2023,13,40,"[29, 30, 31]",31.0
183,15_08_53.edf,5955882,20-06-2023,13,8,"[29, 30, 31]",NaN


## rename eyetracking files 
with assigned sub IDs (found in previous step)


In [84]:
df = df_sum_fil_fil[['filename','assignedID']]
df.head()

,filename,assignedID
2,11_32_33.edf,54.0
4,12_03_33.edf,34.0
6,10_54_48.edf,12.0
10,16_23_07.edf,11.0
11,17_12_44.edf,52.0


In [ ]:
for index, row in df.iterrows():
        subID =  '%02d' % int(row['assignedID'])
        fn_old = os.path.join(pati_eye,row['filename'])
        fn_new = os.path.join(pati_eye_new,f"sub-{subID}.edf")
        os.rename(fn_old, fn_new) # Renames a file and can also move it to a new location if the new name includes a different path.
        print(f"Renamed {fn_old} to {fn_new}")
        
# works fine

## old, beginnings

In [134]:
file_path = os.path.join(pati_eye,'13_58_41.edf' ) #fns[3]
mod_time = os.path.getmtime(file_path)

mod_time = datetime.fromtimestamp(mod_time)## _%H-%M-%S
print(file_path + ' created ' + mod_time.strftime('%d-%m-%Y') )

/Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk_backup/13_58_41.edf created 06-02-2023


In [117]:
file_year = mod_time.year
file_month = mod_time.month
file_day = mod_time.day

matching_row = df[df['date'].apply(lambda x: (x.year == file_year) and (x.month == file_month) and (x.day == file_day))]


In [126]:
fns_2days = []
fns_unmatched = []

for fn in fns:
    full_fn = os.path.join(pati_eye,fn) 
    mod_time = os.path.getmtime(full_fn)
    mod_time = datetime.fromtimestamp(mod_time)
    matching_row = df[df['date'].apply(lambda x: (x.year == mod_time.year) and (x.month == mod_time.month) and (x.day == mod_time.day))]

    if len(matching_row['subject ID'].values) == 1: # only one match!
        sub_id = matching_row['subject ID'].values[0]
        new_filename = f"sub-{sub_id}_{mod_time.day}-{mod_time.month}-{mod_time.year}.edf"
        full_fn_new = os.path.join(pati_eye_new,new_filename)
        os.rename(full_fn, full_fn_new) # Renames a file and can also move it to a new location if the new name includes a different path.
        print(f"Renamed {full_fn} to {new_filename}")
        
    elif len(matching_row['subject ID'].values) == 0:
        fns_unmatched.append(fn)
    elif len(matching_row['subject ID'].values) == 2:
        fns_2days.append(fn)



Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_05_31.edf to sub-51_16-1-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/10_54_48.edf to sub-12_12-4-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_58_41.edf to sub-1_6-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/16_15_12.edf to sub-1_6-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/16_23_07.edf to sub-11_15-3-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/17_12_44.edf to sub-52_16-2-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/14_29_27.edf to sub-24_13-6-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/10_43_53.edf to sub-59_8-4-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/09_38_25.edf to sub-6_28-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_

In [ ]:
## format ?
# first non-symbolic, then symbolic

In [127]:
fns_2days

['eye002.log',
 '11_32_33.edf',
 '10_49_31.edf',
 '12_03_33.edf',
 '16_20_56.edf',
 'eye003.log',
 '16_12_21.edf',
 'eye007.log',
 '11_16_37.edf',
 '11_02_56.edf',
 '15_46_18.edf',
 '16_10_56.edf',
 '11_15_06.edf',
 '11_42_17.edf',
 '16_03_11.edf',
 '10_10_56.edf',
 '11_44_48.edf',
 '11_02_08.edf',
 '11_11_48.edf',
 '15_31_41.edf',
 '16_03_10.edf',
 '11_21_19.edf',
 '11_00_14.edf',
 '12_12_08.edf',
 '16_12_58.edf',
 '16_54_30.edf',
 '11_33_18.edf',
 '10_01_34.edf',
 '15_53_33.edf',
 '10_31_40.edf',
 '11_12_17.edf',
 '11_23_22.edf',
 '10_49_59.edf',
 '16_15_57.edf',
 '10_18_39.edf',
 '10_52_52.edf',
 '16_28_49.edf',
 '10_40_08.edf',
 '10_40_35.edf',
 '16_06_58.edf',
 '17_03_23.edf',
 'eye009.log',
 '11_11_34.edf',
 '16_37_41.edf',
 '11_31_44.edf',
 '11_04_02.edf',
 '11_21_43.edf',
 '16_02_06.edf',
 'trackerui.log']

In [128]:
import re
from io import StringIO
import argparse
import os.path as op
import os
import subprocess
import gzip

import glob
import pandas as pd
import shutil

In [ ]:


def extract_data(subject, root_folder='/data'):
    subject = int(subject)

    dir = get_subject_folder(subject, root_folder) 

    for run in runs:
        fn = op.join(dir, f'Rs{subject:02d}rn{run:02d}.edf')
        asc_fn = fn.replace('.edf', '.asc')
        gaze_target_fn = fn.replace('.edf', '.gaz.gz')

        # get gaze
        cmd = f'edf2asc  -t -y -z -v -s -vel {fn}'

        subprocess.run(cmd, shell=True)

        with open(asc_fn, 'rb') as asc_file, gzip.open(gaze_target_fn, 'wb') as target_file:
                target_file.writelines(asc_file)

        os.remove(asc_fn)

        # get messages
        cmd = f'edf2asc  -t -y -z -v -e {fn}'
        msg_target_fn = fn.replace('.edf', '.msg.gz')

        subprocess.run(cmd, shell=True)

        with open(asc_fn, 'rb') as asc_file, gzip.open(msg_target_fn, 'wb') as target_file:
                target_file.writelines(asc_file)

        os.remove(asc_fn)